# GISY6400 Capstone - Wildfire Hazard: WUI

Program: wildfire_wui.ipynb  
Programmer: Brian Gauthier  
Purpose: This notebook defines the wildland-urban interface (WUI)  
Date: May 11, 2025

### Import Python Modules

In [1]:
import arcpy
import os
from arcpy.sa import *

### Setup

In [2]:
# Set ArcPy Workspace & overwrite options
arcpy.env.workspace = r"D:\Dropbox\COGS\Capstone\gis\capstone.gdb"
arcpy.env.overwriteOutput = True

arcpy.CheckOutExtension("Spatial")

'CheckedOut'

### Define Variables

In [3]:
# Buildings and forest feature classes from which all outputs originate
buildings_fc = "buildings_point"
forest_fc = "forest_inv"

### Create Filtered Forest Feature Classes (forest type, crown closure, volume and basal area parameters)

In [4]:
# Create valid forest types feature class based on NSDNR forest types:

# 0 natural stand
# 1 treated
# 4 sugarbush
# 5 old field
# 6 windthrow
# 7 dead
# 8 dead 1
# 9 dead 2
# 10 research stand
# 11 seed and orchard
# 12 treated stand
# 13 dead 3
# 14 dead 4
# 15 dead 5
# 16 moose meadow
# 20 plantation
# 39 alders 75%+
# 61 partial depletion
# 62 partial cut
forest_types_fc = "forest_types"
arcpy.analysis.Select(forest_fc, forest_types_fc, "FORNON IN (0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 20, 39, 61, 62)")

# Create valid forest types feature class with <50% CC
forest_50_less_fc = "forest_50_less"
arcpy.analysis.Select(forest_types_fc, forest_50_less_fc, "CRNCL < 50")

# Create valid forest types feature class with >= 50% CC
forest_50_fc = "forest_50"
arcpy.analysis.Select(forest_types_fc, forest_50_fc, "CRNCL >= 50")

# Create valid forest types feature class type with >= 75% CC OR total stand basal area (m2/ha) OR total merch vol (m3/ha)
forest_75_fc = "forest_75"
arcpy.analysis.Select(forest_types_fc, forest_75_fc, "(CRNCL >= 75 OR TOTSTDBA >= 10 OR TOTMRVOL >= 100)")

<Result 'D:\\Dropbox\\COGS\\Capstone\\gis\\capstone.gdb\\forest_75'>

### WUI - Intermix

In [ ]:
# Spatial Join: Count buildings in forest stands that have >=50% CC
f50_bldgs_fc = "f50_bldgs"
arcpy.analysis.SpatialJoin(forest_50_fc, buildings_fc, f50_bldgs_fc, 
                           join_type="KEEP_COMMON", join_operation="JOIN_ONE_TO_ONE")

# Add and calculate a field for building density (structures/km²)
arcpy.management.AddField(f50_bldgs_fc, "building_density", "FLOAT")

arcpy.management.CalculateField(f50_bldgs_fc, "building_density", "!Join_Count! / !Shape_Area! * 1000000", "PYTHON3")

# Make a layer from f50_bldgs_fc to use for selection
arcpy.management.MakeFeatureLayer(f50_bldgs_fc, "f50_bldgs_lyr")

# Select stands with building density >= 6.17 structures/km²
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="f50_bldgs_lyr",
    selection_type="NEW_SELECTION",
    where_clause="building_density >= 6.17")

# Export selected stands (WUI Intermix) to a new feature class
wui_intermix_fc = "wui_intermix"
arcpy.management.CopyFeatures("f50_bldgs_lyr", wui_intermix_fc)

### WUI Interface

In [7]:
# Join buildings feature class to stands with <50% CC
f50less_bldgs_fc = "f50less_bldgs"
arcpy.analysis.SpatialJoin(forest_50_less_fc, buildings_fc, f50less_bldgs_fc, 
                           join_type="KEEP_COMMON", join_operation="JOIN_ONE_TO_ONE")

# Add field for building density (structures/km²) and calculate
arcpy.management.AddField(f50less_bldgs_fc, "building_density", "FLOAT")
arcpy.management.CalculateField(f50less_bldgs_fc, "building_density", 
                                "!Join_Count! / !Shape_Area! * 1000000", "PYTHON3")

# Make a feature layer and select areas with building density >= 6.17
arcpy.management.MakeFeatureLayer(f50less_bldgs_fc, "f50less_bldgs_lyr")
arcpy.management.SelectLayerByAttribute("f50less_bldgs_lyr", "NEW_SELECTION", 
                                        "building_density >= 6.17")

# Dissolve wildland patches (≥75% CC or vol 100 m3 or ba 10 m2) to create larger blobs
forest_75_dissolved_fc = "forest_75_dissolved"
arcpy.management.Dissolve(forest_75_fc, forest_75_dissolved_fc, multi_part="SINGLE_PART")

# Add area field and calculate area
arcpy.management.AddField(forest_75_dissolved_fc, "area_sqm", "DOUBLE")
arcpy.management.CalculateGeometryAttributes(forest_75_dissolved_fc, [["area_sqm", "AREA"]], 
                                             area_unit="SQUARE_METERS")

# Create a feature layer from dissolved forest
arcpy.management.MakeFeatureLayer(forest_75_dissolved_fc, "forest_75_dissolved_lyr")

# Select only features that are >= 5km²
arcpy.management.SelectLayerByAttribute("forest_75_dissolved_lyr", "NEW_SELECTION", "area_sqm >= 5000000")

# Export selected features to a new feature class
large_wildland_fc = "large_wildland"
arcpy.management.CopyFeatures("forest_75_dissolved_lyr", large_wildland_fc)

# Buffer only the large wildland patches
wildland_buffer_fc = "wildland_buffer"
arcpy.analysis.Buffer(large_wildland_fc, wildland_buffer_fc, "2.4 Kilometers")

# Select developed areas (<50% CC, ≥6.17 structures/km²) that intersect the buffer
# This ensures both attribute and spatial filters are honored
arcpy.management.SelectLayerByLocation("f50less_bldgs_lyr", "INTERSECT", wildland_buffer_fc, 
                                       selection_type="SUBSET_SELECTION")

# Export final interface WUI areas
wui_interface_fc = "wui_interface"
arcpy.management.CopyFeatures("f50less_bldgs_lyr", wui_interface_fc)

<Result 'D:\\Dropbox\\COGS\\Capstone\\gis\\capstone.gdb\\wui_interface'>

### Combine WUI Interface and WUI Intermix

In [8]:
# Final merge of Intermix and Interface zones into unified WUI feature class
wui_fc = "wui"
arcpy.management.Merge(["wui_intermix", "wui_interface"], wui_fc)

<Result 'D:\\Dropbox\\COGS\\Capstone\\gis\\capstone.gdb\\wui'>